In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['OMP_NUM_THREADS'] = '3'
os.environ['KMP_BLOCKTIME'] = '1'
import numpy as np
import gc
from tqdm.auto import tqdm
from dataset import get_labelled, get_unlabelled, _preload, _unload, unlabelled_generator
from Commit import CommitFactory
from Model import CommitDiffModelFactory
import tensorflow as tf

In [2]:
##### META PARAMS #####

#Size / depth / length of each individual context/path
CONTEXT_SIZE = 16

#Total number of contexts to sample per bag
BAG_SIZE = 256

#Encoder to use
ENCODER = 0

#Size of fixed length vector to encode to
OUTPUT_SIZE = 512

# Hard limit to filter bags by. Bags which exceed this size won't even be sampled.
# Set to None to turn off
MAX_BAG_SIZE_FILTER = 2048

#Maximum number of labelled commits to train on
MAX_LABELLED_COMMITS = 512

#Maximum number of unlabelled commits to train on
MAX_UNLABELLED_COMMITS = 10000

#Batch size for unsupervised training. Higher = faster/better but more memory required
SIAM_BATCH_SIZE = 64

In [3]:
output_types = (tf.float32, tf.float32)
output_shapes = ((BAG_SIZE, CONTEXT_SIZE), (BAG_SIZE, CONTEXT_SIZE))

dataset = tf.data.Dataset.from_generator(
    unlabelled_generator,
    output_types=output_types,
    output_shapes=output_shapes,
    args=(BAG_SIZE, CONTEXT_SIZE, MAX_UNLABELLED_COMMITS, MAX_BAG_SIZE_FILTER)
)

dataset = dataset.batch(SIAM_BATCH_SIZE)

In [4]:
Commit = CommitFactory(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)
CommitDiffModel = CommitDiffModelFactory(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE, OUTPUT_SIZE=OUTPUT_SIZE)

In [5]:
model = CommitDiffModel(unsupervised_data_size = MAX_UNLABELLED_COMMITS, siam_batch_size = SIAM_BATCH_SIZE)
model.initialize(encoder=ENCODER)

In [ ]:
model.fit_siam_generator(dataset, epochs=8, verbose=1)

Epoch 1/8
157/157 [==============================] - 201s 1s/step - loss: -0.0650
Epoch 2/8
157/157 [==============================] - 205s 1s/step - loss: -0.1008
Epoch 3/8
157/157 [==============================] - 220s 1s/step - loss: -0.1494
Epoch 4/8
 80/157 [==============>...............] - ETA: 1:36 - loss: -0.1865

In [ ]:
_preload(max_commits = MAX_UNLABELLED_COMMITS)

In [ ]:
X_train, X_test, y_train, y_test = get_labelled(BAG_SIZE=BAG_SIZE, CONTEXT_SIZE=CONTEXT_SIZE)

In [ ]:
print("Train set size", len(y_train))
print("Train set split", np.sum(y_train)/len(y_train))
print("Test set size", len(y_test))
print("Test set split", np.sum(y_test)/len(y_test))

In [ ]:
#For memory optimization
_unload()

In [ ]:
model.fit_binary_classification(X_train, np.array(y_train), epochs=8, batch_size=64, verbose=1)

In [ ]:
model.evaluate_binary_classification(X_test, np.array(y_test), verbose=0)